In [8]:
import json
import pandas as pd
from datetime import timedelta
import psycopg2

In [2]:
with open(r'C:\Users\DY\Documents\GitHub\UFCproject\ODDS API\unique_fighter_odds.json', 'r') as file:
    data = json.load(file)
df = pd.DataFrame(data)

In [3]:
# Assuming 'df' is your DataFrame and 'date' is the column with the dates
# Convert the 'date' column to datetime in the original DataFrame
df['date'] = pd.to_datetime(df['date'])

# Create a copy of the DataFrame for modification
df_modified = df.copy()

# Subtract one day in the copied DataFrame
df_modified['date'] = df_modified['date'] - timedelta(days=1)

# Concatenate the original and modified DataFrames
df_combined = pd.concat([df, df_modified])

# Sort the DataFrame if needed (optional)
df_combined = df_combined.sort_values(by='date')

In [4]:
df_combined['date'] = pd.to_datetime(df_combined['date'])

# Convert back to the desired string format (YYYY-MM-DD)
df_combined['date'] = df_combined['date'].dt.strftime('%Y-%m-%d')

In [5]:
df['date'] = pd.to_datetime(df['date'])

# Convert back to the desired string format (YYYY-MM-DD)
df['date'] = df['date'].dt.strftime('%Y-%m-%d')

In [6]:
with open('connection_string.txt') as file:
    conn_string = file.readline()

In [9]:
conn = psycopg2.connect(conn_string)
if conn.closed == 0:
    print("Connection established")

Connection established


In [11]:
cur = conn.cursor()

insert_query = """
    INSERT INTO odds (date, fighter_A, fighter_B, Bookmaker, Odds_A, Odds_B)
    VALUES (%s, %s, %s, %s, %s, %s);
"""
# Prepare a list of tuples from the DataFrame
records_to_insert = [tuple(row) for row in df.to_numpy()]

# Execute the query
try:
    cur.executemany(insert_query, records_to_insert)
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print("Error: %s" % error)
    conn.rollback()